[taiga](https://taiga.edwh.nl/project/remco-ewcore/us/991?milestone=98)
# Init postgress db

In [ ]:
%run work/__init__edwh__new.ipynb

In [ ]:
from pydal import DAL

# Init sqlite tags db

In [ ]:
nieuwe_tags = DAL("sqlite://data/out.sqlite")

# Converteren van excel tags die zijn gestored in de sqlite db naar een dictionary

In [ ]:
sqlite_tags_as_dict = nieuwe_tags.executesql('''
    select * from "2021_22"
''', as_dict=True)
sqlite_tags = {row["gid"]: row for row in sqlite_tags_as_dict}
print(len(sqlite_tags_as_dict))

# Updaten van rijen in de postgress op basis van de sqlite rij

In [ ]:

def update_pg_row(postgres_gid, sqlite_row, replaced_by=None):
    deprecated = sqlite_row["Deprecated"] in (1,'1',True)
    args = dict(
        definition=sqlite_row["Definitie"],
        definition_source=sqlite_row["Bron definitie"],
        search_hints=sqlite_row["Extra zoekwoorden"],
        akas=[_.strip() for _ in sqlite_row["Alternatieve benamingen_; ipv _ !"].split(';')],
        questions=sqlite_row["Vragen horend bij hoofdtag"],
        definitions_of_done=sqlite_row[
            "Definitions of Done horend bij hoofdtag"],
        restrictions=sqlite_row["Wanneer tag wel/niet gebruiken"],
        name=sqlite_row["Hoofdtag"],
        related=[_.strip() for _ in sqlite_row["Relatie GID"].split(';') if _.strip()],
        remarks=(sqlite_row[
                     "Opmerking 1_[initialen]:…_verwijderen want is alternatieve benaming van een andere tag"] + '\n' +
                 sqlite_row["Opmerking 2"]).strip(),
    )
    args = {k: v for k, v in args.items() if v is not None and v != "" and v != [] and v != ['']}
    args['deprecated'] = deprecated
    if replaced_by is not None:
        args['replaced_by'] = replaced_by
    # print(db(db.tag.gid == postgres_gid)._update(**args))
    db(db.tag.gid == postgres_gid).update(**args)
    return deprecated

# Vervangen van elke rij in postgres op basis van welke gid hij heeft(denk aan ["", "geen", uuid])

De gid en id die hij heeft staan in de postgres db onder ["Vervangen door [gid] of _geen_" en "tag-id; hoger is nieuwer"]


In [ ]:
db.rollback()
deprecation_count = 0
for cnt, sqlite_row in enumerate(tqdm(sqlite_tags_as_dict), 1):
    postgres_gid = sqlite_row["gid"].strip()
    # print(repr(postgres_gid))
    deprecated = False
    replaced_by = sqlite_row["Vervangen door [gid] of _geen_"].strip()
    if replaced_by == "":
        # print('blanko')
        deprecated = update_pg_row(postgres_gid, sqlite_row)
    elif replaced_by == "geen":
        # print('geen')
        deprecated = update_pg_row(postgres_gid, sqlite_row)
    else:
        # print('replby:',repr(replaced_by))
        deprecated = update_pg_row(postgres_gid, sqlite_row,
                      replaced_by=replaced_by.replace(' ', '').replace('\n', '').split(';')[0])
    if deprecated:
        deprecation_count+= 1
display(deprecation_count)

In [ ]:
db.commit()

In [ ]:

for tag in [_ for _ in sqlite_tags_as_dict if ';' in _["Vervangen door [gid] of _geen_"]]:
    replaced_by = [_.strip() for _ in tag["Vervangen door [gid] of _geen_"].strip().split(';')]
    for repl in replaced_by:
        print('⚠️', tag["Hoofdtag"], ':', sqlite_tags[repl]['Hoofdtag'])

# Wat moet er gebeuren voor verwijderde tags?

De tags zijn te herkennen aan de depcrated kolom.

Een translate dictionary die voor elke tag ingevuld staat en die de tag vervangt (een ander in geval van deprication) of de tag zelf als er geen vervanging is. Bij een verwijdering zonder vervanging is de value None

1. maak een translate tabel van de oude naar de nieuwe tags:

In [ ]:

def replaced_by(tag):
    if tag.deprecated:
        if tag.replaced_by:
            return tag.replaced_by
        else:
            return None
    else:
        return tag.gid


translate = {tag.gid: replaced_by(tag) for tag in db(db.tag).select()}
display({k:v for k,v in translate.items() if k != v })
display(len({k:v for k,v in translate.items() if k != v }))

In [ ]:

if input('Rollback? y/n') == 'y':
    db.rollback()
    print('rollback')
else:
    print('no rollback')


2. itereer over de artikelen:
   ```
    nieuwe_tags = []
    itereer over de oude tags:
        als de tag deprecated is:
            vervang de tag met de nieuwe tag
    update artikel
   ```


In [ ]:
updated, withheld = 0, 0
for idx, item in enumerate(tqdm(db(db.item.platform == 'SvS').select())):
    prev = item.tags[:]
    item.tags = list({translate.get(tag,tag) for tag in item.tags})

    removed = len([tag for tag in item.tags if tag is None])
    # if removed:
    #   print(f'removing {removed} drepcated tags from {item.name}')
    item.tags = [tag for tag in item.tags if tag]
    if item.tags != prev:
        updated += 1
        # print(f'{item.gid} {prev} -> {item.tags}')
    else:
        withheld += 1
    item.update_record()
print(f'updated {updated} withheld {withheld}')

In [ ]:
deprecated_count = 0
for pgtag in tqdm(db(db.tag).select()):
    pgtag.parents = list({translate[tag] for tag in pgtag.parents or [] if translate[tag]})
    pgtag.children = list({translate[tag] for tag in pgtag.children or [] if translate[tag]})
    deprecated = translate[pgtag.gid] != pgtag.gid
    deprecated_count += 1 if deprecated else 0
    pgtag.update_record()
print(f'deprecated {deprecated_count}')

In [ ]:
db.executesql('''
select count(*), deprecated from tag  group by deprecated
''')

In [ ]:
db.commit()